# Retrodiction Testing

For more information about the testing process check out the write-up here! https://www.gamemodelsfootball.com/articles/gen2

`soccerdata` is necessary to get the league data for testing. If the package isn't installed, run `pip install soccerdata`.

In [1]:
import pandas as pd
import numpy as np

import soccerdata as sd
import statsmodels.api as sm

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

seasons = ['17-18', '18-19', '19-20', '20-21', '21-22', '22-23']

team_replace = {
     "WBA": "West Brom", 
     "FC Koln": "Köln", 
     "Stoke": "Stoke City",                 
     "Atletico": "Atletico Madrid", 
     "Real Betis": "Betis", 
     "Bayern": "Bayern Munich",                                        
     "Deportivo": "La Coruña", 
     "Mainz": "Mainz 05", 
     "SPAL 2013": "SPAL",
     "PSG": "Paris S-G",
     "Swansea": "Swansea City",
     "Hertha Berlin": "Hertha BSC",
     "Leicester": "Leicester City", 
     "Man Utd": "Manchester Utd", 
     "Deportivo Alaves": "Alavés", 
     "Verona": "Chievo", 
     "Borussia M.Gladbach": "M'Gladbach", 
     "Man City": "Manchester City",
     "Borussia Dortmund": "Dortmund", 
     "Eintracht Frankfurt": "Eint Frankfurt",
     "Schalke": "Schalke 04", 
     "Hamburg": "Hamburger SV", 
     "Newcastle": "Newcastle Utd", 
     "RBL": "RB Leipzig",
     "Malaga": "Málaga",
     "Fortuna Duesselford": "Dusseldorf",
     "Parma Calcio 1913": "Parma",
     "Cardiff": "Cardiff City",
     "Real Valladolid": "Valladolid",
     "Nimes": "Nîmes",
     "Nuernberg": "Nürnberg",
     "SD Huesca": "Huesca", 
     "Sheff Utd": "Sheffield Utd",
     "Norwich": "Norwich City",
     "Paderborn": "Paderborn 07",
     "Cadiz": "Cádiz",
     "Leeds": "Leeds United",
     "Arminia Bielefeld": "Arminia",
     "Greuther Feurth": "Greuther Fürth"
}

[09/21/24 11:30:13] INFO     Custom team name replacements loaded from                                ]8;id=28930;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=513360;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_config.py#85\85]8;;\
                             /Users/donaldcayton/soccerdata/config/teamname_replacements.json.                     

                    INFO     Custom league dict loaded from                                          ]8;id=376356;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=114843;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_config.py#194\194]8;;\
                             /Users/donaldcayton/soccerdata/config/league_dict.json.                               

## FIFA Ratings

In [2]:
from rapidfuzz import process, fuzz
pd.options.mode.chained_assignment = None

fifa_ratings = pd.read_csv('data/FifaRatings.csv', low_memory = False)

next_season_outcomes = pd.DataFrame()

for season in seasons[1:]:
    fbref = sd.FBref(leagues = 'Big 5 European Leagues Combined', seasons = season)
    season_stats = fbref.read_team_season_stats("standard").reset_index()
    opp_stats = fbref.read_team_season_stats("standard", opponent_stats = True).reset_index()

    season_outcome = pd.DataFrame({
                    'team': season_stats.team,
                    'league': season_stats.league,
                    '90s': season_stats['Playing Time']['90s'],
                    'G-PK': season_stats['Performance']['G-PK'],
                    'G-PK_against': opp_stats['Performance']['G-PK'],
                    'NP-GD': season_stats['Performance']['G-PK'] - opp_stats['Performance']['G-PK']
                })

    season_outcome = season_outcome.sort_values('team').reset_index(drop = True)

    player_stats = fbref.read_player_season_stats("standard").reset_index()

    player_stats.columns = [''.join(c) for c in player_stats.columns]

    # join on birth year, fuzzy-matched name, league_name, nationality, FIFA 19 ratings for 1819 season
    season_fifa = fifa_ratings[fifa_ratings['fifa_version'] == int(season.split('-')[1])]

    season_fifa.loc[:, 'born'] = season_fifa.loc[:, 'dob'].str.split('-').str[0]
    season_fifa.loc[:, 'born'] = season_fifa.born.astype(int)

    player_stats.loc[:, 'league_name'] =  player_stats.league.str.split('-').str[1]

    def initials_score(name1, name2):
        name1_initials = ''.join([part[0] for part in name1.split()])
        name2_initials = ''.join([part[0] for part in name2.split()])
        return fuzz.ratio(name1_initials, name2_initials)

    def combined_score(name1, name2):
        initials_similarity = initials_score(name1, name2)
        fuzzy_similarity = fuzz.ratio(name1, name2)
        # Combine scores with weights
        return 0.5 * fuzzy_similarity + 0.5 * initials_similarity

    def combined_score_wrapper(name1, name2, **kwargs):
        return combined_score(name1, name2)

    def get_candidates(row, df2, other_columns):
        return df2[
            (df2[other_columns[0]] == row[other_columns[0]])
        ]['short_name'].tolist()

    def find_best_match(row, candidates, scorer=combined_score_wrapper, cutoff=80):
        if candidates:
            match = process.extractOne(row, candidates, scorer=scorer, score_cutoff=cutoff)
            if match:
                return match[0]
        return None

    player_stats['Candidates'] = player_stats.apply(lambda row: get_candidates(row, season_fifa, ['born']), axis=1)
    player_stats['Best_Match'] = player_stats.apply(lambda row: find_best_match(row['player'], row['Candidates']), axis=1)

    pd.set_option('display.max_columns', 200)
    player_stats = player_stats.merge(season_fifa, how = 'left', left_on = ['Best_Match', 'born'], right_on = ['short_name', 'born'])

    player_stats['overall'] = player_stats['overall'].fillna(player_stats['overall'].mean())

    # get minute weighted sum of fifa ratings
    player_stats['total_added'] = player_stats['overall'] * player_stats['Playing Time90s']
    
    # next season
    season_outcome['rating_total'] = player_stats.groupby('team', as_index = False).sum()['total_added']
    season_outcome['season'] = season
    
    next_season_outcomes = pd.concat([season_outcome, next_season_outcomes], axis = 0)

# simple regression on goal differential for this season and next season (but ensure we know how to do it first)
next_season_outcomes = pd.get_dummies(next_season_outcomes, columns=['league'], drop_first=True, dtype = float)

X = np.asarray(next_season_outcomes[['rating_total'] + list(next_season_outcomes.columns[next_season_outcomes.columns.str.startswith('league_')])])
y = np.asarray(next_season_outcomes['NP-GD'])

X = sm.add_constant(X)
X = X.astype(float)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_values = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = sm.OLS(y_train, X_train).fit()
    y_pred = model.predict(X_test)
    
    residuals = (y_test - y_pred) / next_season_outcomes['90s'].iloc[test_index]
    rmse = np.sqrt(np.mean(residuals**2))
    rmse_values.append(rmse)

fifa_rmse = np.mean(rmse_values)
print("Average RMSE across 5-fold cross-validation:", fifa_rmse)

[09/21/24 11:30:20] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=455277;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=306609;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:30:27] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=541824;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=796538;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:30:35] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=472741;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=817649;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:30:43] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=25305;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=912392;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:30:52] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=885908;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=624588;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

Average RMSE across 5-fold cross-validation: 0.6175090681501851


--------
## FotMob

In [3]:
leagues = ['bundesliga', 'la_liga', 'ligue_un', 'premier_league', 'serie_a']

fotmob_ratings = pd.DataFrame()
for league in leagues:
    ratings = pd.read_csv(f"data/fotmob/{league}_fotmob.csv")
    league_dict = {"bundesliga": "Bundesliga",
                   "serie_a": "Serie A",
                   "premier_league": "Premier League",
                   "ligue_un": "Ligue 1",
                   "la_liga": "La Liga"
                  }
    
    ratings['league_name'] = league.replace(league, league_dict[league])
    ratings['Season'] = ratings.Season.str.replace('-', '')
    
    ratings = ratings.rename({'Season': 'season', 'Player': 'player'}, axis = 1)
    
    fotmob_ratings = pd.concat([fotmob_ratings, ratings], axis = 0)

next_season_outcomes = pd.DataFrame()

fotmob_ratings.groupby(['player', 'league_name', 'season'], as_index=False).mean()

for season in seasons[1:]:
    fbref = sd.FBref(leagues='Big 5 European Leagues Combined', seasons=season)
    season_stats = fbref.read_team_season_stats("standard").reset_index()
    opp_stats = fbref.read_team_season_stats("standard", opponent_stats=True).reset_index()

    season_outcome = pd.DataFrame({
        'team': season_stats.team,
        'league': season_stats.league,
        '90s': season_stats['Playing Time']['90s'],
        'G-PK': season_stats['Performance']['G-PK'],
        'G-PK_against': opp_stats['Performance']['G-PK'],
        'NP-GD': season_stats['Performance']['G-PK'] - opp_stats['Performance']['G-PK']
    })

    fotmob_next_season_ratings = fotmob_ratings.copy()
    fotmob_next_season_ratings = fotmob_next_season_ratings[['player', 'season', 'Rating']].groupby(['player', 'season'], as_index=False).mean()
    fotmob_next_season_ratings['season'] = (fotmob_next_season_ratings.season.astype(int) + 101).astype(str)

    season_outcome = season_outcome.sort_values('team').reset_index(drop=True)
    
    player_stats = fbref.read_player_season_stats("standard").reset_index()
    player_stats['league_name'] = player_stats.league.str.split("-").str[1]
    player_stats.columns = [''.join(c) for c in player_stats.columns]

    player_stats = player_stats.merge(fotmob_next_season_ratings, how='left', on=['player', 'season'])
    player_stats['Rating'] = player_stats['Rating'].fillna(player_stats['Rating'].mean())

    player_stats['total_added'] = player_stats['Rating'] * player_stats['Playing Time90s']

    season_outcome['total_added'] = player_stats.groupby('team', as_index=False).sum()['total_added']
    season_outcome['season'] = season

    next_season_outcomes = pd.concat([season_outcome, next_season_outcomes], axis=0)


X = np.asarray(next_season_outcomes[['total_added']])
y = np.asarray(next_season_outcomes['NP-GD'])

X = sm.add_constant(X)
X = X.astype(float)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_values = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = sm.OLS(y_train, X_train).fit()
    
    y_pred = model.predict(X_test)
    
    residuals = (y_test - y_pred) / next_season_outcomes['90s'].iloc[test_index]
    rmse = np.sqrt(np.mean(residuals**2))
    rmse_values.append(rmse)

fotmob_rmse = np.mean(rmse_values)
print("Average RMSE across 5-fold cross-validation:", fotmob_rmse)

[09/21/24 11:31:03] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=753292;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=251351;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:05] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=137666;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=504766;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:06] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=139241;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=956430;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:07] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=487120;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=715895;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:08] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=739504;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=774226;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

Average RMSE across 5-fold cross-validation: 0.6400152668275302


--------
## SofaScore

In [4]:
leagues = ['bundesliga', 'la_liga', 'ligue_un', 'premier_league', 'serie_a']

sofascore_ratings = pd.DataFrame()
for league in leagues:
    ratings = pd.read_csv(f"data/sofascore/{league}_sofascore.csv")
    league_dict = {"bundesliga": "Bundesliga",
                   "serie_a": "Serie A",
                   "premier_league": "Premier League",
                   "ligue_un": "Ligue 1",
                   "la_liga": "La Liga"
                  }
    
    ratings['league_name'] = league.replace(league, league_dict[league])
    ratings['season'] = ratings.season.str.replace('-', '')
    
    sofascore_ratings = pd.concat([sofascore_ratings, ratings], axis = 0)

next_season_outcomes = pd.DataFrame()

for season in seasons[1:]:
    fbref = sd.FBref(leagues='Big 5 European Leagues Combined', seasons=season)
    season_stats = fbref.read_team_season_stats("standard").reset_index()
    opp_stats = fbref.read_team_season_stats("standard", opponent_stats=True).reset_index()

    season_outcome = pd.DataFrame({
        'team': season_stats.team,
        'league': season_stats.league,
        '90s': season_stats['Playing Time']['90s'],
        'G-PK': season_stats['Performance']['G-PK'],
        'G-PK_against': opp_stats['Performance']['G-PK'],
        'NP-GD': season_stats['Performance']['G-PK'] - opp_stats['Performance']['G-PK']
    })

    sofascore_next_season_ratings = sofascore_ratings.copy()
    sofascore_next_season_ratings = sofascore_next_season_ratings[['player', 'season', 'rating']].groupby(['player', 'season'], as_index=False).mean()
    sofascore_next_season_ratings['season'] = (sofascore_next_season_ratings.season.astype(int) + 101).astype(str)

    season_outcome = season_outcome.sort_values('team').reset_index(drop=True)
    
    player_stats = fbref.read_player_season_stats("standard").reset_index()
    player_stats['league_name'] = player_stats.league.str.split("-").str[1]
    player_stats.columns = [''.join(c) for c in player_stats.columns]

    player_stats = player_stats.merge(sofascore_next_season_ratings, how='left', on=['player', 'season'])
    player_stats['rating'] = player_stats['rating'].fillna(player_stats['rating'].mean())

    player_stats['total_added'] = player_stats['rating'] * player_stats['Playing Time90s']

    season_outcome['total_added'] = player_stats.groupby('team', as_index=False).sum()['total_added']
    season_outcome['season'] = season

    next_season_outcomes = pd.concat([season_outcome, next_season_outcomes], axis=0)

X = np.asarray(next_season_outcomes[['total_added']])
y = np.asarray(next_season_outcomes['NP-GD'])

X = sm.add_constant(X)
X = X.astype(float)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_values = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = sm.OLS(y_train, X_train).fit()
    
    y_pred = model.predict(X_test)
    
    residuals = (y_test - y_pred) / next_season_outcomes['90s'].iloc[test_index]
    rmse = np.sqrt(np.mean(residuals**2))
    rmse_values.append(rmse)

sofascore_rmse = np.mean(rmse_values)
print("Average RMSE across 5-fold cross-validation:", sofascore_rmse)

[09/21/24 11:31:11] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=723947;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=325918;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:12] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=242570;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=204887;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:13] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=851919;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=289183;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:14] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=290455;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=157014;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:16] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=766923;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=769987;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

Average RMSE across 5-fold cross-validation: 0.6397153374484134


-------
## VAEP

In [5]:
from unidecode import unidecode
import re

def convert_name(name):
    name_ascii = unidecode(name)
    name_cleaned = re.sub(r'[^a-zA-Z0-9\s-]', '', name_ascii)
    return name_cleaned

In [6]:
all_season_outcomes = pd.DataFrame()

for i, season in enumerate(seasons[1:]):
    vaep = pd.read_csv(f'data/spadl/spadl_{seasons[i]}/adj_vaep_{seasons[i]}.csv', index_col = 'Unnamed: 0')

    fbref = sd.FBref(leagues = 'Big 5 European Leagues Combined', seasons = season)
    season_stats = fbref.read_team_season_stats("standard").reset_index()
    opp_stats = fbref.read_team_season_stats("standard", opponent_stats = True).reset_index()

    season_outcome = pd.DataFrame({
                    'team': season_stats.team,
                    'league': season_stats.league,
                    '90s': season_stats['Playing Time']['90s'],
                    'G-PK': season_stats['Performance']['G-PK'],
                    'G-PK_against': opp_stats['Performance']['G-PK'],
                    'NP-GD': season_stats['Performance']['G-PK'] - opp_stats['Performance']['G-PK']
                })

    season_outcome = season_outcome.sort_values('team').reset_index(drop = True)

    player_stats = fbref.read_player_season_stats("standard").reset_index()

    player_stats['league_name'] = player_stats.league.str.split("-").str[1]
    player_stats.columns = [''.join(c) for c in player_stats.columns]
    
    vaep.team_name = vaep.team_name.replace(team_replace)
    
    vaep = vaep.rename({'player_name': 'player', 'team_name': 'team'}, axis = 1)
    
    player_stats['player'] = player_stats['player'].apply(convert_name)
    vaep['player'] = vaep['player'].apply(convert_name)

    player_stats = player_stats.merge(vaep, how = 'left', on = ['player'])
    player_stats.vaep_rating = player_stats.vaep_rating.fillna(np.mean(player_stats.vaep_rating))
    
    player_stats['total_added'] = player_stats['vaep_rating'] * player_stats['Playing Time90s']

    season_outcome['total_added'] = player_stats.groupby('team_x', as_index = False).sum()['total_added']
    season_outcome['season'] = season
    
    all_season_outcomes = pd.concat([all_season_outcomes, season_outcome], axis = 0)

all_season_outcomes = all_season_outcomes.reset_index(drop=True)


X = np.asarray(all_season_outcomes[['total_added']])
y = np.asarray(all_season_outcomes['NP-GD'] / all_season_outcomes['90s'])

X = sm.add_constant(X)
X = X.astype(float)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_values = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = sm.OLS(y_train, X_train).fit()
    
    y_pred = model.predict(X_test)
    
    residuals = y_test - y_pred
    
    rmse = np.sqrt(np.mean(residuals**2))
    rmse_values.append(rmse)

vaep_rmse = np.mean(rmse_values)
print("Average RMSE across 5-fold cross-validation:", vaep_rmse)

[09/21/24 11:31:18] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=487357;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=315493;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:19] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=409654;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=799381;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:21] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=981754;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=102638;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:22] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=598831;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=47509;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:23] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=379055;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=835848;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

Average RMSE across 5-fold cross-validation: 0.5188237565753758


-------
## Atomic VAEP

In [7]:
all_season_outcomes = pd.DataFrame()

for i, season in enumerate(seasons[1:]):
    vaep = pd.read_csv(f'data/spadl/spadl_{seasons[i]}/adj_atomic_vaep_{seasons[i]}.csv', index_col = 'Unnamed: 0')

    fbref = sd.FBref(leagues = 'Big 5 European Leagues Combined', seasons = season)
    season_stats = fbref.read_team_season_stats("standard").reset_index()
    opp_stats = fbref.read_team_season_stats("standard", opponent_stats = True).reset_index()

    season_outcome = pd.DataFrame({
                    'team': season_stats.team,
                    'league': season_stats.league,
                    '90s': season_stats['Playing Time']['90s'],
                    'G-PK': season_stats['Performance']['G-PK'],
                    'G-PK_against': opp_stats['Performance']['G-PK'],
                    'NP-GD': season_stats['Performance']['G-PK'] - opp_stats['Performance']['G-PK']
                })

    season_outcome = season_outcome.sort_values('team').reset_index(drop = True)

    player_stats = fbref.read_player_season_stats("standard").reset_index()

    player_stats['league_name'] = player_stats.league.str.split("-").str[1]
    player_stats.columns = [''.join(c) for c in player_stats.columns]
    
    vaep.team_name = vaep.team_name.replace(team_replace)
    
    vaep = vaep.rename({'player_name': 'player', 'team_name': 'team'}, axis = 1)
    
    player_stats['player'] = player_stats['player'].apply(convert_name)
    vaep['player'] = vaep['player'].apply(convert_name)

    player_stats = player_stats.merge(vaep, how = 'left', on = ['player'])
    player_stats.vaep_rating = player_stats.vaep_rating.fillna(np.mean(player_stats.vaep_rating))
    
    player_stats['total_added'] = player_stats['vaep_rating'] * player_stats['Playing Time90s']

    season_outcome['total_added'] = player_stats.groupby('team_x', as_index = False).sum()['total_added']
    season_outcome['season'] = season
    
    all_season_outcomes = pd.concat([all_season_outcomes, season_outcome], axis = 0)

all_season_outcomes = all_season_outcomes.reset_index(drop=True)

all_season_outcomes = pd.get_dummies(all_season_outcomes, columns=['league'], drop_first=True, dtype = float)

X = np.asarray(all_season_outcomes[['total_added'] + list(all_season_outcomes.columns[all_season_outcomes.columns.str.startswith('league_')])])
y = np.asarray(all_season_outcomes['NP-GD'] / all_season_outcomes['90s'])

X = sm.add_constant(X)
X = X.astype(float)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_values = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = sm.OLS(y_train, X_train).fit()
    
    y_pred = model.predict(X_test)
    
    residuals = y_test - y_pred
    
    rmse = np.sqrt(np.mean(residuals**2))
    rmse_values.append(rmse)

atomic_vaep_rmse = np.mean(rmse_values)
print("Average RMSE across 5-fold cross-validation:", atomic_vaep_rmse)

[09/21/24 11:31:25] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=21092;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=850086;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:26] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=116977;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=812808;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:27] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=212921;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=152725;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:29] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=347696;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=364230;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:30] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=352278;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=600526;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

Average RMSE across 5-fold cross-validation: 0.5568371778764369


-----
## MBAPPE

In [8]:
mbappe_results = pd.DataFrame()
for i, season in enumerate(seasons[1:]):
    fbref = sd.FBref(leagues = 'Big 5 European Leagues Combined', seasons = season)
    season_stats = fbref.read_team_season_stats("standard").reset_index()
    opp_stats = fbref.read_team_season_stats("standard", opponent_stats = True).reset_index()
    
    season_outcome = pd.DataFrame({
                        'team': season_stats.team,
                        'league': season_stats.league,
                        '90s': season_stats['Playing Time']['90s'],
                        'G-PK': season_stats['Performance']['G-PK'],
                        'G-PK_against': opp_stats['Performance']['G-PK'],
                        'NP-GD': season_stats['Performance']['G-PK'] - opp_stats['Performance']['G-PK']
                    })

    season_outcome = season_outcome.sort_values('team').reset_index(drop = True)
    ratings = pd.read_csv(f'data/mbappe/ratings_{seasons[i]}.csv')
    
    player_stats = fbref.read_player_season_stats("standard").reset_index()
    player_stats.columns = [''.join(c) for c in player_stats.columns]
    
    ratings = ratings.rename({'Player': 'player', 'Born': 'born'}, axis = 1)
    player_stats = player_stats.merge(ratings, how = 'left', on = ['player', 'born'])
    
    player_stats['MBAPPE'] = player_stats['MBAPPE'].fillna(0)
    
    player_stats['total_added'] = player_stats['MBAPPE'] * player_stats['Playing Time90s']
    
    season_outcome['retrodictive_GD'] = player_stats.groupby('team', as_index = False).sum()['total_added']
    
    season_outcome['NP-GD'] = season_outcome['NP-GD'] / season_outcome['90s']
            
    season_outcome['G-PK'] = season_outcome['G-PK'] / season_outcome['90s']
    season_outcome['G-PK_against'] = season_outcome['G-PK_against'] / season_outcome['90s']

    season_outcome['retrodictive_GD'] = season_outcome['retrodictive_GD'] / season_outcome['90s']
    season_outcome['season'] = season
    
    mbappe_results = pd.concat([mbappe_results, season_outcome], axis = 0)

mbappe_results = pd.get_dummies(mbappe_results, columns=['league'], drop_first=True, dtype = float)

X = np.asarray(mbappe_results[['retrodictive_GD'] + list(mbappe_results.columns[mbappe_results.columns.str.startswith('league_')])])
y = np.asarray(mbappe_results['NP-GD'])

X = sm.add_constant(X)
X = X.astype(float)

kf = KFold(n_splits=5, shuffle=True, random_state=29)
rmse_values = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = sm.OLS(y_train, X_train).fit()
    
    y_pred = model.predict(X_test)
    
    residuals = y_test - y_pred
    
    rmse = np.sqrt(np.mean(residuals**2))
    rmse_values.append(rmse)

mbappe_rmse = np.mean(rmse_values)
print("Average RMSE across 5-fold cross-validation:", mbappe_rmse)

[09/21/24 11:31:36] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=550328;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=970176;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:37] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=281476;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=381083;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:38] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=758524;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=256077;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:39] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=556094;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=851960;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

[09/21/24 11:31:40] INFO     Saving cached data to /Users/donaldcayton/soccerdata/data/FBref         ]8;id=311854;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=845523;file:///Users/donaldcayton/anaconda3/lib/python3.11/site-packages/soccerdata/_common.py#263\263]8;;\

Average RMSE across 5-fold cross-validation: 0.4881476952403695


------
## Final Results

In [9]:
results = pd.DataFrame({
    "Metric": ["FIFA Ratings", "FotMob", "SofaScore", "VAEP", "Atomic VAEP", "MBAPPE"],
    "RMSE": [fifa_rmse, fotmob_rmse, sofascore_rmse, vaep_rmse, atomic_vaep_rmse, mbappe_rmse]
})

results = results.sort_values("RMSE", ascending = True)

In [10]:
results

,Metric,RMSE
5,MBAPPE,0.488148
3,VAEP,0.518824
4,Atomic VAEP,0.556837
0,FIFA Ratings,0.617509
2,SofaScore,0.639715
1,FotMob,0.640015
